In [1]:
import random
import matplotlib.pyplot as plt
import numpy as np
import umap
from tensorflow.keras import backend as K
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *

#import Lambda
from tensorflow.keras.layers import Lambda

import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator,  load_img
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense
from sklearn.model_selection import train_test_split

c:\Users\levy\miniconda3\envs\keras\lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\levy\miniconda3\envs\keras\lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\levy\miniconda3\envs\keras\lib\site-packages\umap\distances.py:1086: NumbaDepreca

In [13]:
# load mnist dataset from keras

(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [37]:
# train a triplet neural network on mnist dataset
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Input, Lambda
#import concatenate
from tensorflow.keras.layers import Concatenate
# define triplet loss function, receive vectors of anchor, positive and negative

def triplet_loss(y_true, y_pred):
    anchor_out = y_pred[:, 0:100]
    positive_out = y_pred[:, 100:200]
    negative_out = y_pred[:, 200:300]

    pos_dist = K.sum(K.abs(anchor_out - positive_out), axis=1)
    neg_dist = K.sum(K.abs(anchor_out - negative_out), axis=1)

    probs = K.softmax([pos_dist, neg_dist], axis=0)

    return K.mean(K.abs(probs[0]) + K.abs(1.0 - probs[1]))


# define triplet neural network

def triplet_network(input_shape):
    input_layer = Input(input_shape)
   

    # use custom cnn model
    X = Conv2D(32, 3, activation='relu')(input_layer)
    X = MaxPool2D(2)(X)
    X = Conv2D(64, 3, activation='relu')(X)
    X = MaxPool2D(2)(X)
    X = Flatten()(X)
    X = Dense(100, activation='relu')(X)
    model = Model(input_layer, X)


    triplet_model_a = Input((28,28,1))
    triplet_model_p = Input((28,28,1))
    triplet_model_n = Input((28,28,1))
    triplet_model_out = Concatenate()([model(triplet_model_a), model(triplet_model_p), model(triplet_model_n)])

    triplet_model = Model([triplet_model_a, triplet_model_p, triplet_model_n], triplet_model_out)
    return triplet_model


# define triplet neural network model

triplet_model = triplet_network((28, 28, 1))
triplet_model.compile(optimizer='adam', loss=triplet_loss, metrics=['accuracy'])
triplet_model.summary()


Model: "model_18"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_43 (InputLayer)          [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 input_44 (InputLayer)          [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 input_45 (InputLayer)          [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 model_17 (Functional)          (None, 100)          178916      ['input_43[0][0]',               
                                                                  'input_44[0][0]',        

In [38]:
# define triplet generator

def triplet_generator(x, y, batch_size):
    while True:
        x_anchor = []
        x_positive = []
        x_negative = []
        for i in range(batch_size):

            # anchor and positive must be from the same class
            random_label = random.choice(np.unique(y))
            (anchor, positive) = random.sample(list(x[y == random_label]), 2)

            # negative must be from different class

            while True:
                negative_label = random.choice(np.unique(y))
                if negative_label != random_label:
                    break
            negative = random.choice(list(x[y == negative_label]))

            x_anchor.append(anchor)
            x_positive.append(positive)
            x_negative.append(negative)

        x_anchor = np.asarray(x_anchor)
        x_positive = np.asarray(x_positive)
        x_negative = np.asarray(x_negative)
        yield [x_anchor, x_positive, x_negative], []


In [39]:
triplet_model.compile(loss = triplet_loss, optimizer='adam')

In [41]:
triplet_model.fit_generator(triplet_generator(x_train, y_train, 32), steps_per_epoch=100, epochs=3)

C:\Users\levy\AppData\Local\Temp\ipykernel_18512\1102308065.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  triplet_model.fit_generator(triplet_generator(x_train, y_train, 32), steps_per_epoch=100, epochs=3)


Epoch 1/3


ValueError: in user code:

    File "c:\Users\levy\miniconda3\envs\keras\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\levy\miniconda3\envs\keras\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\levy\miniconda3\envs\keras\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\levy\miniconda3\envs\keras\lib\site-packages\keras\engine\training.py", line 995, in train_step
        self._validate_target_and_loss(y, loss)
    File "c:\Users\levy\miniconda3\envs\keras\lib\site-packages\keras\engine\training.py", line 959, in _validate_target_and_loss
        raise ValueError(

    ValueError: No loss found. You may have forgotten to provide a `loss` argument in the `compile()` method.
